In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import glob

from utils.file_dialog_utils import pick_image_cv2, pick_folder

def load_templates_with_names(folder_path):
    """
    Loads templates and keeps their filenames.
    Returns a list of tuples: [('name1.png', img1), ('name2.png', img2), ...]
    """
    # Get all png files
    paths = sorted(glob.glob(os.path.join(folder_path, "*.png")))
    
    template_data = []
    for p in paths:
        img = cv2.imread(p)
        if img is not None:
            filename = os.path.basename(p)
            template_data.append((filename, img))
            
    return template_data

def find_best_match_with_progress(target_gray, template_data):
    """
    Scans the target image against the list of templates.
    Prints progress exactly like the original script.
    """
    best_match = {
        "score": -1.0,
        "name": None,
        "location": None,
        "width": 0,
        "height": 0
    }

    total_templates = len(template_data)

    print(f"Starting match process on {total_templates} templates...")
    print("-" * 60)

    # Loop with index using enumerate()
    for index, (name, tmpl_bgr) in enumerate(template_data):
        
        # Convert template to gray for matching
        tmpl_gray = cv2.cvtColor(tmpl_bgr, cv2.COLOR_BGR2GRAY)
        
        # 1. Match
        res = cv2.matchTemplate(target_gray, tmpl_gray, cv2.TM_CCOEFF_NORMED)
        
        # 2. Get Score
        _, max_val, _, max_loc = cv2.minMaxLoc(res)

        # 3. Update Best Match
        if max_val > best_match["score"]:
            h, w = tmpl_gray.shape
            best_match["score"] = max_val
            best_match["name"] = name
            best_match["location"] = max_loc
            best_match["width"] = w
            best_match["height"] = h

        # 4. Print Progress
        # Format: [Current/Total] Filename | Score | Best So Far
        print(f"[{index + 1}/{total_templates}] {name} | Score: {max_val:.4f} | Best: {best_match['score']:.4f}")

    return best_match

# --- MAIN EXECUTION ---

_ , image_path = pick_image_cv2(title="Select Main Image for Template Matching")   
template_folder = pick_folder(title="Select Template Folder")

# 1. Load Main Image
main_image = cv2.imread(image_path)
if main_image is None:
    print(f"Error: Could not load image at {image_path}")
else:
    main_gray = cv2.cvtColor(main_image, cv2.COLOR_BGR2GRAY)

    # 2. Load Templates
    templates = load_templates_with_names(template_folder)

    if not templates:
        print("No templates found in the folder.")
    else:
        # 3. Run Matching
        result = find_best_match_with_progress(main_gray, templates)

        print("-" * 60)
        print(f"FINAL RESULT: {result['name']} with score {result['score']:.4f}")

        # 4. Visualization
        if result['location']:
            top_left = result['location']
            w, h = result['width'], result['height']
            bottom_right = (top_left[0] + w, top_left[1] + h)
            
            # Draw rectangle
            cv2.rectangle(main_image, top_left, bottom_right, (0, 255, 0), 2)
            
            # Add label text
            text = f"{result['name']} ({result['score']:.2f})"
            cv2.putText(main_image, text, (top_left[0], top_left[1] - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            plt.figure(figsize=(10, 8))
            plt.imshow(cv2.cvtColor(main_image, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.show()